In [37]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
# будем использовать полный набор после отладки на тестовом.
# Полный набор: 
# movies_all = pd.read_csv('../../datasets/movies.csv')
# ratings_all = pd.read_csv('../../datasets/ratings.csv')
# tags_all = pd.read_csv('../../datasets/tags.csv')
# Тестовый набор
movies_all = pd.read_csv('movies.csv')
ratings_all = pd.read_csv('ratings.csv')
tags_all = pd.read_csv('tags.csv')

## Подготовка train-test-split
Объединяем все данные в один большой dataframe с уникальными строками по паре пользователь-фильм,  
делим ее на обучающую и тестовую выборки

In [35]:
# ! Выполнение длится долго (больше минуты) на больших данных
# Все, что дал каждый пользователь в udf.
# Считаем, что пользователя нет, если он не добавил тэг или не проставил рейтинг
tags_all_temp = tags_all.drop(['movieId', 'timestamp'], axis=1, inplace=False)

udf = ratings_all.join(tags_all_temp.set_index('userId'), on='userId') 
# Рейтинг есть у всех фильмов с тэгами?
print(udf[udf.tag.notnull() & udf.rating.isnull()])
umdf = udf.join(movies_all.set_index('movieId'), on='movieId')

Empty DataFrame
Columns: [userId, movieId, rating, timestamp, tag]
Index: []


In [38]:
umdf.head()

,userId,movieId,rating,timestamp,tag,title,genres
0,1,1,4.0,964982703,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,NaN,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,NaN,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [39]:
# Выделение тестовой выборки

trdf, testdf, ytr, ytest = train_test_split(umdf.drop(['rating'], axis=1, inplace=False), 
umdf['rating'], 
test_size=0.33, random_state=42)

In [40]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [42]:
movie_genres = [change_string(g) for g in trdf.drop_duplicates('movieId').genres.values]
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=7)

In [45]:
# Проверка на произвольных жанрах
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)
res = neigh.kneighbors(X_tfidf2, return_distance=True)
# res
trdf.iloc[res[1][0]]

,userId,movieId,timestamp,tag,title,genres
25174,177,2355,1435890634,feel-good,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
73172,474,296,979179939,racism,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
93136,599,1464,1519119509,travolta,Lost Highway (1997),Crime|Drama|Fantasy|Film-Noir|Mystery|Romance
75180,474,52952,1201832936,Stephen King,This Is England (2006),Drama
73546,474,1573,1100291472,football,Face/Off (1997),Action|Crime|Drama|Thriller
74125,474,3683,1137521116,slasher,Blood Simple (1984),Crime|Drama|Film-Noir
74938,474,8695,1089638396,George Bernard Shaw,"Bachelor and the Bobby-Soxer, The (1947)",Comedy


In [51]:
# Очень неоптимально! медленно даже на небольших данных из-за дублирования всех тэгов всех фильмов
# Использовать другую структуру данных
# и/или переписать цикл на векторыне операции

# Подготовка тэгов - приведение к тому же формату, что и в жанрах
# movies_with_tags.shape
movies_with_tags = trdf.dropna(subset=['tag'], axis = 0,inplace=False)
# movies_with_tags.head()
print(movies_with_tags[movies_with_tags.tag.isnull()])
tag_strings_list = []
movies_list = []
trdf['tags'] = ""

for movie, group in tqdm(movies_with_tags.groupby('title')):
    m_tags = '|'.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values])
    tag_strings_list.append(m_tags)
    trdf.loc[trdf.movieId == group.movieId.values[0],['tags']] = m_tags
    movies_list.append(movie)

# tag_strings_list[:5]

Empty DataFrame
Columns: [userId, movieId, timestamp, tag, title, genres]
Index: []


  0%|          | 0/6358 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
# Построение графа ближайших соседей
movie_tags = [change_string(g) for g in movies.tags.values]
count_vect_tags = CountVectorizer()
X_train_counts_tags = count_vect_tags.fit_transform(movie_tags)
tfidf_transformer_tags = TfidfTransformer()
X_train_tfidf_tags = tfidf_transformer_tags.fit_transform(X_train_counts_tags)
neigh_tags = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh_tags.fit(X_train_tfidf_tags)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=7)

In [8]:
# Проверка на произвольных тэгах
test = change_string('pixar|pixar|fun')

predict = count_vect_tags.transform([test])
X_tfidf2 = tfidf_transformer_tags.transform(predict)

res = neigh_tags.kneighbors(X_tfidf2, return_distance=True)
# res
movies.iloc[res[1][0]]

,movieId,title,genres,tags
2271,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,animation|Disney|Pixar|insects|KevinSpacey|opp...
3028,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Pixar|sequelbetterthanoriginal|abandonment|ani...
14509,72356,Partly Cloudy (2009),Animation|Children|Comedy|Fantasy,Pixar|shortfilm|Pixar|shortfilm|Pixar|memasa's...
4791,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,funny|Pixar|Comedy|funny|Pixar|animated|animat...
41760,157296,Finding Dory (2016),Adventure|Animation|Comedy,adventure|animation|pixar|animation|computeran...
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,animated|buddymovie|Cartoon|cgi|comedy|compute...
11048,45517,Cars (2006),Animation|Children|Comedy,redemption|villainnonexistentornotneededforgoo...


Как я понял задачу.  
Нужно оценить рейтинг для фильмов из тестовой выборки двумя способами.

1. по TFIDF жанра и тэгов (без относительно пользователей) построить оценку фильмам с помощью регрессии

Задание "построить рекомендации" в этом случае трактуется так. 
Мы выдадим рейтинги фильмам, которые никто ранее не оценивал, по их тэгам и жанрам. 
Для этого берем ближайших соседей к каждому фильму, усредняем их рейтинг - это значение и задаем как оценку рейтинга.
Пользователь (вне зависимости от предпочтений) получит эти новые фильмы отсортированные в порядке убывания сгенерированного рейтинга.
То есть получит "средние" рекомендации.

2. использовать для вычисления рейтинга (регрессией) средние оценки пользователя и фильма (а также, возможно, другие метрики)

Задание "построить рекомендации" в этом случае трактуется так. 
Получается большой вектор со строками "пользователь-фильм" длины равной длине вектора ratings. На таком векторе мы должны обучить регрессию и для такого же тестового вектора предсказать оценки.
В качестве фич можно использовать такие:
* средняя оценка фильма
* средняя оценка пользователя
* средняя оценка фильмов этого жанра (всеми пользователями)
* средняя оценка фильмов этого жанра (этим пользователем)
* (под сомнением) средняя оценка этого фильма любителями этого жанра (любитель - пользователь оценивший больше фильмов этого жанра, чем другие пользователи в среднем) - не факт, что поможет, т.к. больше всего оценок генерил персонал сервиса, как было показано в материалах к лекции


